In [1]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import numpy as np
import os
from skimage.feature import hog
from skimage.color import rgb2gray

Chargement des données d'entraînement


In [6]:
# Chargement des données des patchs normalisés
pos_patch_fs = [
    os.path.splitext(f)[0]
    for f in os.listdir(os.path.join("local_data", "4_normalized_patches", "pos"))
    if f.endswith(".jpg")
]
neg_patch_fs = [
    os.path.splitext(f)[0]
    for f in os.listdir(os.path.join("local_data", "4_normalized_patches", "neg"))
    if f.endswith(".jpg")
]

pos_patchs = []
neg_patchs = []

for f in pos_patch_fs:
    try:
        patch = plt.imread(
            os.path.join("local_data", "4_normalized_patches", "pos", f"{f}.jpg")
        )
        pos_patchs.append(patch[
            :, :, 0
        ])  # le channel grayscale est dupliqué sur les 3 chanaux RGB (on en isole 1)

    except FileNotFoundError:
        continue

for f in neg_patch_fs:
    try:
        patch = plt.imread(
            os.path.join("local_data", "4_normalized_patches", "neg", f"{f}.jpg")
        )
        neg_patchs.append(patch[
            :, :, 0
        ])  # le channel grayscale est dupliqué sur les 3 chanaux RGB (on en isole 1)

    except FileNotFoundError:
        continue

nb_patchs_tot = len(pos_patchs) + len(neg_patchs)
print(len(pos_patchs))
print(len(neg_patchs))

1204
6020


In [5]:
# Chargement des images de test
imgs_test_f = [
    os.path.splitext(f)[0]
    for f in os.listdir(
        os.path.join("data", "test")
    )
    if f.endswith(".jpg")
]

imgs_test = {}
imgs_test_rgb = {}

for f in imgs_test_f:
    try:
        img = plt.imread(
            os.path.join(
                "data", "test", f"{f}.jpg"
            )
        )
        imgs_test[f] = rgb2gray(img)
        imgs_test_rgb[f] = img
    except FileNotFoundError:
        continue

print(len(imgs_test))

82


In [7]:
y_train = np.hstack(
    [np.ones((1, len(pos_patchs))), np.zeros((1, len(neg_patchs)))]
).flatten()

patchs_train = np.array(pos_patchs + neg_patchs)
print(patchs_train.shape)
print(y_train.shape)

(7224, 128, 72)
(7224,)


In [8]:
# Extraction des features sur les patchs d'entraînement
# TODO : y placer les paramètres optimisés


def HOG_extractor(patchs):
    if len(patchs) == 0:
        return []
    first_features = hog(patchs[0])  # valeurs par défaut
    features = np.zeros(
        shape=(len(patchs), first_features.shape[0]), dtype=first_features.dtype
    )
    for i, patch in enumerate(patchs):
        features[i] = hog(patch)

    return features


X_train = HOG_extractor(patchs_train)
print(X_train.shape)
print(X_train.dtype)

(7224, 7938)
float64


In [9]:
# Entraînement du modèle
# TODO : y placer les paramètres optimisés
# probability=True pour pouvoir utiliser .predict_proba(X) dessus (rajoute pas mal de temps d'entraînement en vrai)

clf = SVC(kernel="poly", probability=True)
clf.fit(X_train, y_train)

SVC(kernel='poly', probability=True)

Autres trucs utiles


In [11]:
stats = {}
with open(
    os.path.join("local_data", "4_normalized_patches", "stats.txt"),
    "r",
    encoding="utf-8",
) as f:
    for line in f:
        line = line.strip()
        if not line or "=" not in line:
            continue

        key, value = line.split("=", 1)

        try:
            val = float(value)
            if val.is_integer():
                val = int(val)
            stats[key] = val
        except ValueError:
            pass

# Extraction dans des variables individuelles (optionnel)
min_ratio = stats.get("min_ratio")
max_ratio = stats.get("max_ratio")
moy_ratio = stats.get("moy_ratio")
min_scale = stats.get("min_scale")
max_scale = stats.get("max_scale")

print(min_ratio)
print(max_ratio)
print(moy_ratio)
print(min_scale)
print(max_scale)

0.42758620689655175
0.9832214765100671
0.5726107997574511
107
967


Explication des variables utiles à la détection


In [12]:
### Essentiels

imgs_test       # liste python des images de test (qui sont déjà grayscale, chaque image est une numpy array 2D)
imgs_test_rgb   # même chose mais en RGB (pour visualisation)

clf  # Le classifieur SVC(kernel="poly") entraîné avec les features extraites grâce à un HOG sur tous les patchs d'entraînement

### Limites de formes des fenêtres de découpe

min_scale  # facteur de scale minimum mesuré dans les bbox d'annotations sur les images positives (c'est-à-dire la longueur du rectangle)
max_scale  # facteur de scale max

min_ratio  # ratio minimum mesuré (rapport entre largeur/hauteur), un ratio petit fait que la fenêtre apparâit plus "carrée"
max_ratio  # ratio maximum mesuré

# Exemple pour obtenir les dimensions de la plus grande fenêtre de test :
# (max_ratio*max_scale, max_scale) : (width, height), la fenêtre apparaît comme un rectangle "debout"
# il faudrait aussi tester le cas inverse :
# (max_scale, max_ratio*max_scale) : (width, height), la fenêtre apparaît comme un rectangle "allongé"


### Traitement des patchs
HOG_extractor  # fonction qui prend une liste ou array de patchs normalisés et extrait les features (pour fabriquer X_test pour soumission à la prédiction du classifieur)

<function __main__.HOG_extractor(patchs)>

In [21]:
import importlib

import utils.detection
importlib.reload(utils.detection)
detect_ecocup = utils.detection.detect_ecocup

test_windows = {} # dictionnaire {"num_img": liste des fenêtres détectées avec p>0.5}
test_scores = {}  # dictionnaire {"num_img": liste des scores de confiance des fenêtres détectées}

for img_name, img in imgs_test.items():
    print(f"\n---------Traitement de l'image {img_name}---------\n")
    all_windows, all_scores = detect_ecocup(
        img, clf, HOG_extractor, min_ratio, max_ratio, min_scale, max_scale,
        px_step=35, scales_nb=10, ratios_nb=5, confidence_threshold=0.5
    )
    test_windows[img_name] = all_windows.tolist()
    test_scores[img_name] = all_scores.tolist()

# Sauvegarde des résultats de détection dans des jsons
import json
with open(os.path.join("test_data", "test_windows.json"), "w") as f:
      json.dump(test_windows, f)
with open(os.path.join("test_data", "test_scores.json"), "w") as f:
      json.dump(test_scores, f)


---------Traitement de l'image 0000---------

Temps setup : 0.0
Nombre d'itérations : 50

	Itération 1 : h=0107 | w=0045
	Temps sliding_window x2 (2886 généré): 0.004036664962768555
	Temps de traitement pour 2886 : 0.5307772159576416
	Temps d'extraction de features pour 2886 : 4.2225635051727295
	Temps de prédiction pour 2886 : 12.480963945388794
	Fenêtres gardées : 2 sur 2886 (p>=0.5)
	Meilleur score de l'itération : 0.913285009636576 pour [[490 210]
 [535 317]]
	Temps cumulé : 17.238341331481934

	Itération 2 : h=0202 | w=0086
	Temps sliding_window x2 (2660 généré): 0.0029909610748291016
	Temps de traitement pour 2660 : 0.8198337554931641
	Temps d'extraction de features pour 2660 : 4.081993818283081
	Temps de prédiction pour 2660 : 12.076321363449097
	Fenêtres gardées : 1 sur 2660 (p>=0.5)
	Meilleur score de l'itération : 0.6137753333925393 pour [[420 280]
 [506 482]]
	Temps cumulé : 34.219481229782104

	Itération 3 : h=0298 | w=0127
	Temps sliding_window x2 (2368 généré): 0.0028107